### The Abstraction: The Process
进程是操作系统提供给用户的最基本的抽象.进程就是一个运行的程序.程序本身是死的,就是disk上的一堆指令以及一些静态数据.os调用这些二进制文件,使它们运行,使程序变得有用.  
同一时间常常有多个程序在运行,即使只有一个cpu,问题来了,os如何虚拟cpu,多个cpu?实际上,在运行一个进程时,os中断并运行另一个,同一时间运行多个cpu的技术称为时间分片 time sharing-这是关键.  
实现cpu的虚拟化,既需要底层的机器又需要高层的算法.例如,在底层我们需要实现cpu时间分片(time sharing),在高层,我们需要实现程序间的上下文转换(context switch). 在最高层,我们需要实现任务调度(scheduling policy).

#### 进程的抽象：The Abstraction: A Process
进程就是运行的程序－我们通过它在运行中接触操作系统各个部分(machine state)来总结它．
1. 最显要的机器状态是内存,指令在内存中,读写的数据也在内存中,进程私有的内存称为地址空间(address space).
2. 另一个重要的是寄存器(registers) 通过读写寄存器,进程被cpu执行.其中program counter (PC)程序计数器告诉我们执行哪道指令,(stack pointer)栈指针,(frame pointer)帧指针用来管理函数参数,本地变量和返回地址.
3. 最后是i/o,进程当前打开了什么文件.

#### 进程的api
1. Create os创建进程
2. Destroy os销毁进程
3. Wait os要求进程等待
4. Miscellaneous Control 一些其他操作,如挂起,复位 suspend aprocess (stop it from running for a while) and then resume it (continue it running).
5. Status os返回进程信息(运行时间,内部状态)

#### 进程如何创建
1. os将位于硬盘上代码和静态数据(初始化变量) 载入内存--进程的地址空间.现代os只在code需要的时候调用它(lazily).
2. 一旦程序被载入内存,os分配内存做为程序的运行时栈(run-time stack),例如c语言的本地变量,函数参数和返回地址;os还会初始化这些参数等待.
3. os会为程序的堆(heap)分配内存.在c语言中,堆是动态分配的数据,malloc()请求空间,free()释放请求空间.heap用做链表,hash表等数据结构的存储.
4. os还会做一些i/o方面的初始化工作,在unix,每个进程都会默认有三个文件描述符,file descriptors,(标准输出,标准输入,标准错误)stdin,stdout,stderr.
5. 最后,os开始在代码入口运行程序 main().
![Screenshot from 2017-04-08 21-40-49.png](https://ooo.0o0.ooo/2017/04/08/58e8e87aa5a0e.png)

#### 进程的状态
进程有三种状态:
* 运行(Running) 进程在执行指令
* 就绪(Ready) 进程准备运行,但os选择不在那时运行
* 堵塞(block) 进程已经进行了某些操作,但必须等待一些事件发生后,才能继续.比如i/o操作,i/o开始初始化后,进程被堵塞,直到i/o完成.

![Screenshot from 2017-04-08 22-38-13.png](https://ooo.0o0.ooo/2017/04/08/58e8f5eb2ba87.png)
一个典型的进程操作:
![Screenshot from 2017-04-08 22-45-00.png](https://ooo.0o0.ooo/2017/04/08/58e8f77e481be.png)

#### os的数据结构
os是一个程序,有一些关键的数据结构,例如为了跟踪进程的状态,os 将保存一些进程list来存储就绪的进程  
下面介绍mit的xv6的数据结构:
1. register context 将会保存一个停止进程的寄存器状态;当重新载入这些寄存器状态,os就能恢复运行的进程.
```
// 8个寄存器状态
struct context {
int eip;
int esp;
int ebx;
int ecx;
int edx;
int esi;
int edi;
int ebp;
};
```
2. 进程状态
```
enum proc_state {UNUSED,EMBRYQ,SLEEPING,RUNNABLE,RUNNING,ZOMBIE};
```
进程被创建后有初始状态,有最终状态(被退出,但还为清除--僵尸进程),僵尸状态是为了父进程能调用子进程的返回值,当父进程执行完毕,它最后调用`wait()` 等待子进程完成,并且告诉操作系统,它可以被清除.

3. 进程
```
struct proc {
char *mem;  //内存地址
uint sz;    //内存大小
char *kstack; //栈底位置
enum proc_state state;//进程状态
int pid;    //进程id
struct proc *parent; //父进程
void *chan; //非零则sleeping
int killed; //非零则killed
struct file *ofile[NOFILE];//打开的文件
struct inode *cwd;//当前目录
struct context context;//切换到此处去执行进程
struct trapfram *tf;//用于当前中断的栈
}
```

question
1.  Run the program with the following flags: ./process-run.py -l 5:100,5:100. What should the CPU utilization be (e.g., the percent of time the CPU is in use?)  
100%
2. Now run with these flags: ./process-run.py -l 4:100,1:0. These flags specify one process with 4 instructions (all to use the CPU), and one that simply issues an I/O and waits for it to be done. How long does it take to complete both processes?  
Stats: Total Time 10
Stats: CPU Busy 5 (50.00%)
Stats: IO Busy  4 (40.00%)
3. Now switch the order of the processes: ./process-run.py -l 1:0,4:100. What happens now? Does switching the order matter?
Stats: Total Time 6
Stats: CPU Busy 5 (83.33%)
Stats: IO Busy  4 (66.67%)  
在i/o进行操作时,进程被堵塞,另一个进程开始运行,充分利用了时间.
4. We’ll now explore some of the other flags. One important flag is -S,which determines how the system reacts when a process issues an I/O. With the flag set to SWITCH ON END, the system will NOT switch to another process while one is doing I/O, instead waiting until the process is completely finished. What happens when you run the following two processes, one doing I/O and the other doing CPU work?
Stats: Total Time 9
Stats: CPU Busy 5 (55.56%)
Stats: IO Busy  4 (44.44%)  
同3差不多,cpu在i/o时等待.
5. Now, run the same processes, but with the switching behavior set to switch to another process whenever one is WAITING for I/O. What happens now?
Stats: Total Time 6
Stats: CPU Busy 5 (83.33%)
Stats: IO Busy  4 (66.67%)  
可见主要时间都节省在i/o上
6. One other important behavior is what to do when an I/O completes.With -I IO RUN LATER, when an I/O completes, the proccess that issued it is not necessarily run right away; rather, whatever was running at the time keeps running. What happens when you run this combination of processes?  
Stats: Total Time 27
Stats: CPU Busy 18 (66.67%)
Stats: IO Busy  12 (44.44%)  
I/o操作也需要cpu,在其他进程运行时,它被堵塞.
7. Now run the same processes, but with -I IO RUN IMMEDIATE set,which immediately runs the process that issued the I/O. How does this behavior differ? Why might running a process that just completed an I/O again be a good idea?  
Stats: Total Time 18
Stats: CPU Busy 18 (100.00%)
Stats: IO Busy  12 (66.67%)
初始化i/o需要cpu,其他时间不需要,因此i/o操作集中的话,就不用在其他进程结束后,等待